In [4]:
import numpy as np
import scipy.integrate as itg
import scipy.linalg as lag
import matplotlib.pyplot as plt
import sympy as sy
import pandas as pd


## Calculating functions

In [5]:
def calculate_system(
    transfer_coefs, input_flows, procs_names, land, material, print_screen=True
):
    # retrieve the internal processes (where transformation happen) from the proc_names database
    proc_names_int = procs_names.loc[
        procs_names["Proc_type"] == "internal", "Proc_name"
    ]

    # get the number of internal processes
    nproc = proc_names_int.nunique()

    # extract the flows for the land
    flow_df_unk = transfer_coefs.loc[transfer_coefs["Country"] == land]

    # extract the flows for the material
    flow_df_unk = flow_df_unk.loc[flow_df_unk["Material"] == material]
    flow_df_unk = flow_df_unk.reset_index(
        drop=True
    )  # drop and reset the index for future poining.

    # Get the number of unknown flos
    nint_fl = len(flow_df_unk)

    # extract input information for land and material
    input_unk = input_flows.loc[transfer_coefs["Country"] == land]
    input_unk = input_unk.loc[input_unk["Material"] == material]

    # create empty matrices based on the size of the unknown flows to solve Ax = b where A is tc_matrix and b is knowns
    tc_matrix = np.zeros((nint_fl, nint_fl))
    knowns = np.zeros(nint_fl)

    # Populate the A and the b matrices
    count = 0  # this counts the processes which should equal nproc
    count2 = 0  # this counts the additional unkowns that are required to solve the system should equal nint_fl-nproc
    tc_lines = (
        []
    )  # list containing tcs and receiving processes info used to extract information for the transfer coefficient based equations
    known_add = (
        []
    )  # list containing known flows used to extract information for the transfer coefficient based equations
    sources = (
        []
    )  # list containing source flows used to extract information for the transfer coefficient based equations
    for process in proc_names_int:
        indpos = flow_df_unk.loc[
            flow_df_unk["ProcessGen"] == process
        ].index  # get indexes of each destination process
        if print_screen:
            print("These are the indeces of the processes")
            print(indpos)
        tc_s = flow_df_unk.loc[
            flow_df_unk["ProcessGen"] == process
        ]  # subset data for each destination process

        # impute A matrix based on process mass balancing
        for j in range(len(tc_s) - 1):
            tc_lines.append(
                [
                    j + count2,
                    tc_s.iloc[j]["Transfer_Coef"],
                    tc_s.iloc[j]["ProcessGen"],
                    tc_s.iloc[j]["ProcessTo"],
                ]
            )  # Populate tc_lines list
        count2 += len(tc_s)
        for i in indpos:
            tc_matrix[count, i] = 1.0  # populate output flows

        indpos = flow_df_unk.loc[
            flow_df_unk["ProcessTo"] == process
        ].index  # Get index information of the source processes
        for i in indpos:
            tc_matrix[count, i] = -1.0  # populate input flows
        sources.append([process, indpos])

        knowns[count] = input_unk.loc[
            input_unk["toProc"] == process, "value"
        ].sum()  # populate b matrix
        known_add.append(
            [process, input_unk.loc[input_unk["toProc"] == process, "value"].sum()]
        )  # popluate known_add list

        count += 1

    # populate the transfer coefficient based balances e.g. flow2 = tc_1_2*flow1
    count2 = 0
    for line in tc_lines:
        tc_matrix[
            nproc + count2, line[0]
        ] = 1.0  # populate the left side of the equality

        for kline in known_add:
            if line[2] == kline[0]:
                knowns[nproc + count2] = (
                    kline[1] * line[1]
                )  # populate the right side fo the equality

        for sline in sources:
            if sline[0] == line[2]:
                for ind in sline[1]:
                    tc_matrix[nproc + count2, ind] = -line[
                        1
                    ]  # populate unknown flows on the A matrix
        count2 += 1

    if print_screen:
        print("\nThese are the known flows (b vector)")
        print(knowns)  # uncomment to debug or to see the final b matrix
        print("\nThis is the technological Matrix (A matrix)")
        print(tc_matrix)  # uncomment to debug or to see the A matrix
    # invert the Ax = b matrix and find the solution for x
    solution = lag.solve(tc_matrix, knowns)

    flowdf = flow_df_unk[["ProcessGen", "ProcessTo"]].copy()
    flowdf["values"] = solution

    return [solution, flowdf]


## Input parameters

In [6]:
PS = True
material = "Total"
location = "EU"
excel_ws = "../data/Test_sheet.xlsx"

## Import information from Excel Sheets

In [7]:
# Transfer coefficient tab
transfer_coefs = pd.read_excel(
    excel_ws, sheet_name="trans_coeff", index_col=None, header=0
)

# Input Flow tab
input_flows = pd.read_excel(
    excel_ws, sheet_name="input_flows", index_col=None, header=0
)

# Processes in the system tab
procs_names = pd.read_excel(excel_ws, sheet_name="procs", index_col=None, header=0)


flows, flowdf = calculate_system(
    transfer_coefs, input_flows, procs_names, location, material, PS
)
flowdf.to_csv(location + "-" + material + ".csv", index=False)
print(flows)

material = "Total"
flows, flowdf = calculate_system(
    transfer_coefs, input_flows, procs_names, location, material, PS
)

flowdf.to_csv(location + "-" + material + ".csv", index=False)
print(flows)

These are the indeces of the processes
Index([0, 1, 2], dtype='int64')
These are the indeces of the processes
Index([3], dtype='int64')
These are the indeces of the processes
Index([4, 5, 6], dtype='int64')
These are the indeces of the processes
Index([7, 8, 9], dtype='int64')
These are the indeces of the processes
Index([10, 11, 12], dtype='int64')
These are the indeces of the processes
Index([13, 14, 15], dtype='int64')
These are the indeces of the processes
Index([16, 17, 18], dtype='int64')
These are the indeces of the processes
Index([19, 20, 21, 22], dtype='int64')

These are the known flows (b vector)
[130.2    0.   245.6   58.2    0.     0.     0.     0.     6.51  78.12
  49.12 122.8   40.74  11.64   0.     0.     0.     0.     0.     0.
   0.     0.     0.  ]

This is the technological Matrix (A matrix)
[[ 1.    1.    1.   -1.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    1.    0.    0.